In [1]:
from tripadvisor import Tripadvisor

hotel_id = "9793120"

tripadvisor = Tripadvisor()
# hotels = tripadvisor.get_hotels()
hotel = tripadvisor.hotel_detail(hotel_id)
print(hotel)

{'data': [{'location_id': '9793120', 'name': "D'Salvatore Art & Boutique Hotel", 'latitude': '-7.783526', 'longitude': '110.42571', 'num_reviews': '79', 'timezone': 'Asia/Jakarta', 'location_string': 'Yogyakarta, Jawa', 'photo': {'images': {'small': {'width': '150', 'url': 'https://media-cdn.tripadvisor.com/media/photo-l/0a/bc/88/a0/getlstd-property-photo.jpg', 'height': '150'}, 'thumbnail': {'width': '50', 'url': 'https://media-cdn.tripadvisor.com/media/photo-t/0a/bc/88/a0/getlstd-property-photo.jpg', 'height': '50'}, 'original': {'width': '2736', 'url': 'https://media-cdn.tripadvisor.com/media/photo-o/0a/bc/88/a0/getlstd-property-photo.jpg', 'height': '1824'}, 'large': {'width': '550', 'url': 'https://media-cdn.tripadvisor.com/media/photo-s/0a/bc/88/a0/getlstd-property-photo.jpg', 'height': '367'}, 'medium': {'width': '250', 'url': 'https://media-cdn.tripadvisor.com/media/photo-f/0a/bc/88/a0/getlstd-property-photo.jpg', 'height': '167'}}, 'is_blessed': True, 'uploaded_date': '2016-03

In [2]:
import json

print(json.dumps(hotel, indent=4))

{
    "data": [
        {
            "location_id": "9793120",
            "name": "D'Salvatore Art & Boutique Hotel",
            "latitude": "-7.783526",
            "longitude": "110.42571",
            "num_reviews": "79",
            "timezone": "Asia/Jakarta",
            "location_string": "Yogyakarta, Jawa",
            "photo": {
                "images": {
                    "small": {
                        "width": "150",
                        "url": "https://media-cdn.tripadvisor.com/media/photo-l/0a/bc/88/a0/getlstd-property-photo.jpg",
                        "height": "150"
                    },
                    "thumbnail": {
                        "width": "50",
                        "url": "https://media-cdn.tripadvisor.com/media/photo-t/0a/bc/88/a0/getlstd-property-photo.jpg",
                        "height": "50"
                    },
                    "original": {
                        "width": "2736",
                        "url": "https://med

In [ ]:
from tripadvisor import Tripadvisor

hotel_id = "20226478"

tripadvisor = Tripadvisor()
review = tripadvisor.hotel_review(hotel_id, 0)

In [ ]:
from tripadvisor import Tripadvisor

hotel_id = "20226478"

tripadvisor = Tripadvisor()
review = tripadvisor.hotel_review(hotel_id, 900)

In [ ]:
temp_review = review

for data in review['data']:
    try:
        print(data['published_date'])
        print(int(data['travel_date'].split('-')[0]))
    except:
        pass

In [ ]:
import json

# print(type(review['paging']['total_results']))
# print(len(review['data']))
print(json.dumps(review['data'], indent=7))
# print(len(review['data']))

In [ ]:
import json

beneran_review = [data for data in review['data'] if 'ad_context' not in data]
len(beneran_review)

In [ ]:
review2 = tripadvisor.hotel_review(hotel_id, 20)

In [ ]:
print(json.dumps(review2['data'][0], indent=7))

# COLLECT HOTEL DETAIL (COMPLETED)

In [ ]:
from tripadvisor import Tripadvisor
import mysql.connector
import time
import config
import os
import json

hotel_data_dir = './hotel_data'

# initiate mysql
db = init_db()
cursor = get_cursor(db)

# select all hotel list that have not been parsed
hotel_list = db_select(db, "SELECT * FROM hotel_list WHERE hotel_detail_parsed=0 LIMIT 6000")

# instantiate TripAdvisor object
tripadvisor = Tripadvisor()

# iterate per hotel to get hotel detail then change hotel_detail_parsed to True
for hotel in hotel_list:
    hotel_id, hotel_link, hotel_detail_parsed, hotel_review_parsed = hotel

    # create hotel dir if not exist
    hotel_id_dir = os.path.join(hotel_data_dir, hotel_id)
    if not os.path.exists(hotel_id_dir):
        os.makedirs(hotel_id_dir)

    # get hotel detail from tripadvisor api
    hotel_detail = tripadvisor.hotel_detail(hotel_id)

    # store hotel_detail data to database and local file
    if 'errors' in hotel_detail.keys():
            print(f'can\'t parse hotel with id {hotel["hotel_id"]}')
    else:
        #store and update database
        cursor.execute(
            'INSERT IGNORE INTO hotel_detail (hotel_id, hotel_detail) VALUES (%s, %s)',
            (hotel_id, json.dumps(hotel_detail))
        )
        cursor.execute(
            f'UPDATE hotel_list SET hotel_detail_parsed=1 WHERE hotel_id="{hotel_id}"'
        )
        db.commit()

        #store to local file
        hotel_detail_path = os.path.join(hotel_id_dir, f'{hotel_id}_hotel_detail.json') # hotel_detail local file path
        with open(hotel_detail_path, 'w') as json_file:
            json.dump(hotel_detail, json_file, indent=7)
    time.sleep(5)
    
    
def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            database = config.configSQLCrawler.DATABASE,
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

def db_select(con, query):
    try:
        cursor = get_cursor(con)
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(e)
    finally:
        cursor.close()



# COLLECT HOTEL REVIEW

In [ ]:
from tripadvisor import Tripadvisor
import pymongo
from pymongo import MongoClient
import mysql.connector
import time
import config
import os
import json
import random


def get_all_review(hotel_id):
    tripadvisor = Tripadvisor()
    
    offset = 0
    all_hotel_review = list()
    while True:
        hotel_review_raw = tripadvisor.hotel_review(hotel_id, offset)
        hotel_review_data = [data for data in hotel_review_raw['data'] if 'ad_context' not in data]
        total_result = int(hotel_review_raw['paging']['total_results'])
        if len(hotel_review_data) != 50:
            diff = total_result - len(all_hotel_review)
            if diff > 50:
                raise Exception('REVIEW COUNT NOT 50, SOMETHING WRONG')
        all_hotel_review += hotel_review_data
        offset += 50
        print(f'\rCurrent review count: {len(all_hotel_review):<6} | Total results: {total_result:<6}', end='')
        if len(all_hotel_review) == total_result:
            break
        time.sleep(random.uniform(5.0, 10.0))
        continue
    return all_hotel_review


def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            database = config.configSQLCrawler.DATABASE,
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

def db_select(con, query):
    try:
        cursor = get_cursor(con)
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(e)
    finally:
        cursor.close()


mongo_cluster = MongoClient('mongodb://localhost:27017')
mongo_db = mongo_cluster['tripadvisor']
mongo_collection = mongo_db['hotel_review']

hotel_data_dir = './hotel_data'

# initiate mysql
db = init_db()
cursor = get_cursor(db)

# select all hotel list that have not been parsed
hotel_list = db_select(db, "SELECT * FROM hotel_list WHERE hotel_review_parsed=0 LIMIT 44")


# iterate per hotel to get hotel review then change hotel_review_parsed to True
for hotel in hotel_list:
    hotel_id, hotel_link, hotel_detail_parsed, hotel_review_parsed = hotel

    # create hotel dir if not exist
    hotel_id_dir = os.path.join(hotel_data_dir, hotel_id)
    if not os.path.exists(hotel_id_dir):
        os.makedirs(hotel_id_dir)

    # get hotel review from tripadvisor api
    hotel_reviews = get_all_review(hotel_id)
    
    # insert data into mongodb
    # mongo_post = dict()
    # mongo_post['_id'] = hotel_id
    # mongo_post['data'] = hotel_reviews
    # mongo_collection.insert_one(mongo_post)

    #store and update database
    cursor.execute(
        'INSERT IGNORE INTO hotel_review (hotel_id, hotel_reviews) VALUES (%s, %s)',
        (hotel_id, json.dumps(hotel_reviews))
    )
    cursor.execute(
        f'UPDATE hotel_list SET hotel_review_parsed=1 WHERE hotel_id="{hotel_id}"'
    )
    db.commit()

    #store to local file
    hotel_review_path = os.path.join(hotel_id_dir, f'{hotel_id}_hotel_review.json') # hotel_review local file path
    with open(hotel_review_path, 'w') as json_file:
        json.dump(hotel_reviews, json_file, indent=7)
    time.sleep(5)
    




# UPDATE FILE

In [ ]:
from tripadvisor import Tripadvisor
import pymongo
from pymongo import MongoClient
import mysql.connector
import time
import config
import os
import json
import random
import shutil

def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            database = config.configSQLCrawler.DATABASE,
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

def db_select(con, query):
    try:
        cursor = get_cursor(con)
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(e)
    finally:
        cursor.close()

hotel_data_dir = './hotel_data'
destination_folder = './hotel_data/archived'

# initiate mysql
db = init_db()
cursor = get_cursor(db)

hotel_list = db_select(db, "SELECT hotel_id FROM hotel_list")
hotel_list = [hotel[0] for hotel in hotel_list]

current_list = os.listdir(hotel_data_dir)

for hotel_folder in current_list:
    if hotel_folder not in hotel_list:
        dir_to_move = os.path.join(hotel_data_dir, hotel_folder)
        shutil.move(dir_to_move, destination_folder)


# CREATE FOLDER IN HOTEL_DATA


# ALTER JSON FILE AND UPLOAD TO MYSQL SERVER (ARCHIVED)

In [ ]:
import mysql.connector
import config
import json




def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            database = config.configSQLCrawler.DATABASE,
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

def db_select(con, query):
    try:
        cursor = get_cursor(con)
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(e)
    finally:
        cursor.close()
        
destination_path = './hotel_data/hotel_list.json'

db = init_db()
cursor = get_cursor(db)

hotel_list = db_select(db, "SELECT hotel_id FROM hotel_review")
hotel_list = [hotelid[0] for hotelid in hotel_list]

with open('hotel_list.json') as json_file:
    json_data = json.load(json_file)
    hotel_json_list = [hotel['hotel_id'] for hotel in json_data]

for hotel in hotel_list:
    if hotel not in hotel_json_list:
        cursor.execute(f"DELETE FROM hotel_review WHERE hotel_id={hotel}")
        db.commit()

# with open('hotel_list.json') as json_file:
#     json_data = json.load(json_file)
    
#     for hotel in json_data:
#         if hotel['hotel_id'] not in hotel_list:
#             cursor.execute(
#                 "INSERT IGNORE INTO hotel_list (hotel_id, hotel_link, hotel_detail_parsed, hotel_review_parsed) VALUES (%s, %s, %s, %s);",
#                 (hotel['hotel_id'], hotel['link'], False, False )
#                 )
#             db.commit()



# MIGRATE DATA FROM LOCAL TO MONGODB (ARCHIVED)


In [ ]:
import pymongo
from pymongo import MongoClient
import json
import os

hotel_data_dir = './hotel_data'
hotel_detail_paths = [os.path.join(hotel_data_dir,dirr,f'{dirr}_hotel_detail.json') for dirr in os.listdir(hotel_data_dir)]

cluster = MongoClient('mongodb://localhost:27017')
db = cluster['tripadvisor']
collection = db['hotel_detail']

for detail in hotel_detail_paths:
    with open(detail) as json_file:
        json_data = json.load(json_file)
        new_data = dict()
        new_data['_id'] = json_data['data'][0]['location_id']
        new_data['data'] = json_data['data'][0]
        collection.insert_one(new_data)
       






In [ ]:
import pymongo
from pymongo import MongoClient
import json
import os


hotel_data_dir = './hotel_data'
hotel_review_paths = [os.path.join(hotel_data_dir,dirr,f'{dirr}_hotel_review.json') for dirr in os.listdir(hotel_data_dir) if os.path.exists(os.path.join(hotel_data_dir,dirr,f'{dirr}_hotel_review.json'))]

cluster = MongoClient('mongodb://localhost:27017')
db = cluster['tripadvisor']
collection = db['hotel_review']

for hotel_review_path in hotel_review_paths:
    with open(hotel_review_path) as json_file:
        review_data = json.load(json_file)
        for i,review in enumerate(review_data):
            new_data = dict()
            new_data['_id'] = review['id']
            new_data['hotel_id'] = review['location_id']
            new_data['data'] = review
            collection.insert_one(new_data)
# # for detail in hotel_detail_paths:
#     with open(detail) as json_file:
#         json_data = json.load(json_file)
#         new_data = dict()
#         new_data['_id'] = json_data['data'][0]['location_id']
#         new_data['data'] = json_data['data']
#         collection.insert_one(new_data)

# EXPLORE

In [2]:
import pymongo
from pymongo import MongoClient
import json
import os
import config
import mysql.connector


def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            # database = config.configSQLCrawler.DATABASE,
            database = 'data_skripsi',
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

db = init_db()
cursor = get_cursor(db)

hotel_data_dir = './hotel_data'
hotel_detail_paths = [os.path.join(hotel_data_dir,dirr,f'{dirr}_hotel_detail.json') for dirr in os.listdir(hotel_data_dir)]

for hotel_detail in hotel_detail_paths:
    with open(hotel_detail) as hotel_detail_data:
        hotel_detail_dict = json.load(hotel_detail_data)['data'][0]
        
        hotel_id = hotel_detail_dict['location_id']
        hotel_name = hotel_detail_dict['name']
        latitude = hotel_detail_dict['latitude']
        longitude = hotel_detail_dict['longitude']
        num_reviews = hotel_detail_dict['num_reviews']
        location_string = hotel_detail_dict['location_string']
        raw_ranking = hotel_detail_dict['raw_ranking']
        ranking_geo = hotel_detail_dict['ranking_geo']
        ranking_geo_id = hotel_detail_dict['ranking_geo_id']
        ranking_position = hotel_detail_dict['ranking_position']
        ranking_denominator = hotel_detail_dict['ranking_denominator']
        ranking_category = hotel_detail_dict['ranking_category']
        sub_category_type = hotel_detail_dict['subcategory_type']
        rating = hotel_detail_dict['rating']
        is_closed = hotel_detail_dict['is_closed']
        try:
            price = hotel_detail_dict['price']
        except:
            price = ''
        hotel_class = hotel_detail_dict['hotel_class']
        hotel_class_attribution = hotel_detail_dict['hotel_class_attribution']
        postalcode = hotel_detail_dict['address_obj']['postalcode']

        sql = "INSERT INTO hotel_list (hotel_id, hotel_name, latitude, longitude, num_reviews, location_string, raw_ranking, ranking_geo, ranking_geo_id, ranking_position, ranking_denominator, ranking_category, sub_category_type,  rating, is_closed, price, hotel_class, hotel_class_attribution, postalcode) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        data = (hotel_id, hotel_name, latitude, longitude, num_reviews, location_string, raw_ranking, ranking_geo, ranking_geo_id, ranking_position, ranking_denominator, ranking_category, sub_category_type,  rating, is_closed, price, hotel_class, hotel_class_attribution, postalcode)
        cursor.execute(sql, data)
        db.commit()

In [1]:
import pymongo
from pymongo import MongoClient
import json
import os
import config
import mysql.connector


def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            # database = config.configSQLCrawler.DATABASE,
            database = 'data_skripsi',
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

db = init_db()
cursor = get_cursor(db)

hotel_data_dir = './hotel_data'
hotel_reviews_paths = [os.path.join(hotel_data_dir,dirr,f'{dirr}_hotel_review.json') for dirr in os.listdir(hotel_data_dir)]

num_of_reviews = 0
for hotel_reviews in hotel_reviews_paths:
    with open(hotel_reviews) as hotel_reviews_data:
        hotel_reviews = json.load(hotel_reviews_data)
        for review in hotel_reviews:
            hotel_id = review['location_id']
            review_id = review['id']
            lang = review['lang']
            published_date = review['published_date']
            travel_date = review['travel_date']
            type = review['type']
            rating = review['rating']
            helpful_votes = review['helpful_votes']
            title = review['title']
            text = review['text']
            try:
                username = review['user']['username']
            except:
                username = ''
            machine_translated = review['machine_translated']
            machine_translatable = review['machine_translatable']

            sql = 'INSERT INTO hotel_review (hotel_id, review_id, lang, published_date, travel_date, type, rating, helpful_votes, title, text, username, machine_translated, machine_translatable) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
            data = (hotel_id, review_id, lang, published_date, travel_date, type, rating, helpful_votes, title, text, username, machine_translated, machine_translatable)
            cursor.execute(sql, data)
            db.commit()
            num_of_reviews += 1
            print(f'\rNum of reviews: {num_of_reviews:<8}', end='')

Num of reviews: 598699  